In [ ]:
!nvidia-smi

Thu Jul 29 06:39:46 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8    28W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 15287, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 15287 (delta 0), reused 1 (delta 0), pack-reused 15283
Receiving objects: 100% (15287/15287), 13.69 MiB | 882.00 KiB/s, done.
Resolving deltas: 100% (10377/10377), done.


In [ ]:
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

/content/darknet
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:946:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1147:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
           

In [ ]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [ ]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=1@classes=1@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=1@classes=1@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=1@classes=1@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=18@filters=18@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=18@filters=18@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=18@filters=18@' cfg/yolov3_training.cfg

In [ ]:
!mkdir "/mydrive/yolov3"

mkdir: cannot create directory ‘/mydrive/yolov3’: File exists


In [ ]:
!echo "Koala" > data/obj.names
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data
!mkdir data/obj

In [ ]:
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2021-07-29 06:44:14--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  22.9MB/s    in 7.5s    

2021-07-29 06:44:22 (20.7 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



In [ ]:
!unzip /mydrive/yolov3/images.zip -d data/obj

Archive:  /mydrive/yolov3/images.zip.zip
  inflating: data/obj/1.jpg          
  inflating: data/obj/1.txt          
  inflating: data/obj/10.jpg         
  inflating: data/obj/10.txt         
  inflating: data/obj/11.jpg         
  inflating: data/obj/11.txt         
  inflating: data/obj/12.jpeg        
  inflating: data/obj/12.txt         
  inflating: data/obj/13.jpg         
  inflating: data/obj/13.txt         
  inflating: data/obj/14.jpg         
  inflating: data/obj/14.txt         
  inflating: data/obj/15.jpg         
  inflating: data/obj/15.txt         
  inflating: data/obj/16.jpg         
  inflating: data/obj/16.txt         
  inflating: data/obj/17.jpg         
  inflating: data/obj/17.txt         
  inflating: data/obj/2.jpg          
  inflating: data/obj/2.txt          
  inflating: data/obj/3.jpg          
  inflating: data/obj/3.txt          
  inflating: data/obj/4.jpg          
  inflating: data/obj/4.txt          
  inflating: data/obj/5.jpg          
  inflati

In [ ]:
import glob
import os
import re

txt_file_paths = glob.glob(r"data/obj/*.txt")
for i, file_path in enumerate(txt_file_paths):
    with open(file_path, "r") as f_o:
        lines = f_o.readlines()

        text_converted = []
        for line in lines:
            print(line)
            numbers = re.findall("[0-9.]+", line)
            print(numbers)
            if numbers:
                text = "{} {} {} {} {}".format(0, numbers[1], numbers[2], numbers[3], numbers[4])
                text_converted.append(text)
                print(i, file_path)
                print(text)
        with open(file_path, 'w') as fp:
            for item in text_converted:
                fp.writelines("%s\n" % item)

0 0.633984 0.656250 0.130469 0.070833

['0', '0.633984', '0.656250', '0.130469', '0.070833']
0 data/obj/3.txt
0 0.633984 0.656250 0.130469 0.070833
0 0.524609 0.714583 0.327344 0.093056

['0', '0.524609', '0.714583', '0.327344', '0.093056']
1 data/obj/9.txt
0 0.524609 0.714583 0.327344 0.093056
0 0.343359 0.590972 0.260156 0.134722

['0', '0.343359', '0.590972', '0.260156', '0.134722']
2 data/obj/8.txt
0 0.343359 0.590972 0.260156 0.134722
0 0.221094 0.707639 0.112500 0.081944

['0', '0.221094', '0.707639', '0.112500', '0.081944']
3 data/obj/13.txt
0 0.221094 0.707639 0.112500 0.081944
0 0.320703 0.604167 0.091406 0.108333

['0', '0.320703', '0.604167', '0.091406', '0.108333']
4 data/obj/16.txt
0 0.320703 0.604167 0.091406 0.108333
0 0.319922 0.622917 0.117969 0.056944

['0', '0.319922', '0.622917', '0.117969', '0.056944']
5 data/obj/4.txt
0 0.319922 0.622917 0.117969 0.056944
0 0.565625 0.647917 0.087500 0.054167

['0', '0.565625', '0.647917', '0.087500', '0.054167']
6 data/obj/14.txt

In [ ]:
import glob
images_list = glob.glob("data/obj/*.jpg")
print(images_list)

['data/obj/2.jpg', 'data/obj/15.jpg', 'data/obj/14.jpg', 'data/obj/3.jpg', 'data/obj/1.jpg', 'data/obj/7.jpg', 'data/obj/13.jpg', 'data/obj/10.jpg', 'data/obj/9.jpg', 'data/obj/17.jpg', 'data/obj/4.jpg', 'data/obj/11.jpg', 'data/obj/8.jpg', 'data/obj/5.jpg', 'data/obj/6.jpg', 'data/obj/16.jpg']


In [ ]:
file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 

In [ ]:
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.267558), count: 3, class_loss = 1525.982300, iou_loss = 2.997070, total_loss = 1528.979370 
 total_bbox = 3300, rewritten_bbox = 0.000000 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.000000), count: 1, class_loss = 70.409103, iou_loss = 0.000000, total_loss = 70.409103 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.000000), count: 1, class_loss = 276.225189, iou_loss = 0.000000, total_loss = 276.225189 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.271895), count: 4, class_loss = 1526.569214, iou_loss = 4.825439, total_loss = 1531.394653 
 total_bbox = 3304, rewritten_bbox = 0.000000 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.000000), count: 1, class_loss = 69.899338, iou_loss = 0.000000, total_loss = 69.899338 
v3 (